## To do

### Merge do dataset de AQI com o dataset de meteorologia

### Visualização de dados (gráficos, box plots, matriz de correlação, outliers)
### Teste de hipóteses
### Feature Engineering

### Prever com algoritmos o AQI baseado em dados meteorológicos

## Se possível

### Interpolação noutras coordenadas
### Data augmentation

In [ ]:
import pandas as pd 

In [ ]:
# Read the data again

df = pd.read_csv('datasets/FINAL2.csv')

In [ ]:
# Find the coordinates to interpolate

import numpy as np

# Define the coordinates
coordinates = [
    [41.5549, -8.4067],
    [41.653315, -8.58779],
    [41.650791, -8.43569],
    [41.449721999999994, -8.296389]
]

# Convert the coordinates to a numpy array
coordinates_array = np.array(coordinates)

# Calculate the mean of the latitude and longitude
mean_latitude = np.mean(coordinates_array[:, 0])
mean_longitude = np.mean(coordinates_array[:, 1])

# Print the medium point
print("Medium Point Latitude:", mean_latitude)
print("Medium Point Longitude:", mean_longitude)

# 41.57718199999999,-8.43164225 - ZONA NOVA ARCADA
# https://www.google.com/maps/place/41%C2%B034'37.9%22N+8%C2%B025'53.9%22W/@41.5772226,-8.4318715,53m/data=!3m1!1e3!4m4!3m3!8m2!3d41.577182!4d-8.4316422

Medium Point Latitude: 41.57718199999999
Medium Point Longitude: -8.43164225


In [ ]:
# calculate AQI for each row
def calculateAQI(data):

    # define breakpoints
    breakpoints = [
        {
            "pollutant": "no2",
            "conc": [0, 50, 100, 200, 400, 1000],
            "aqi": [0, 50, 100, 150, 200, 300, 400],
        },
        {
            "pollutant": "pm10",
            "conc": [0, 20, 40, 70, 100, 200],
            "aqi": [0, 50, 100, 150, 200, 300, 400],
        },
        {
            "pollutant": "o3",
            "conc": [0, 54, 70, 85, 105, 200],
            "aqi": [0, 50, 100, 150, 200, 300, 400],
        },
    ]

    # calculate AQI for each pollutant
    def aqiForConcentration(pollutant, conc):
        bp = next(bp for bp in breakpoints if bp["pollutant"] == pollutant)
        i = next(i for i, c in enumerate(bp["conc"]) if c > conc) - 1
        cLow = bp["conc"][i]
        cHigh = bp["conc"][i + 1]
        aqiLow = bp["aqi"][i]
        aqiHigh = bp["aqi"][i + 1]
        return round(((aqiHigh - aqiLow) / (cHigh - cLow)) * (conc - cLow) + aqiLow)

    # calculate AQI for each row
    for index, row in data.iterrows():
        aqi = None
        for pollutant in ["no2", "pm10", "o3"]:
            if row[pollutant] is not None:
                aqi = aqiForConcentration(pollutant, row[pollutant])
                break
        data.at[index, "AQI"] = aqi
    
    return data
braga_air_data = calculateAQI(braga_air_data)
# drop columns no2, pm10, o3
braga_air_data = braga_air_data.drop(columns=["no2", "pm10", "o3"])
braga_air_data.head()

In [5]:
from scipy.interpolate import Rbf

filtered_df = df[df['parameter'].isin(['pm1', 'pm25', 'pm10','no2'])]

coordinates = filtered_df[['latitude', 'longitude']].values
values = filtered_df['value'].values

# Perform IDW interpolation
rbf = Rbf(coordinates[:, 0], coordinates[:, 1], values)
interpolated_value = rbf(41.577929249999995, -8.40629225)

# Print the interpolated value
print("Interpolated Value:", interpolated_value)